In [1]:
using Flight.FlightCore.Systems
using Flight.FlightCore.Plotting

using Flight.FlightPhysics
using Flight.FlightComponents
using Flight.FlightAircraft.C172FBW
using Flight.FlightAircraft.C172

using UnPack
using ControlSystems
using RobustAndOptimalControl
using ComponentArrays
using LinearAlgebra

In [ ]:
ac = Cessna172FBWBase(NED()) |> System #linearization requires NED kinematics

#use default trim condition
design_point = C172.TrimParameters()

P_lss_lon = Control.LinearStateSpace(ac, design_point; model = :lon);
P_nss_lon = named_ss(P_lss_lon);

x_labels_lon = keys(P_lss_lon.x0) |> collect
y_labels_lon = keys(P_lss_lon.y0) |> collect
u_labels_lon = keys(P_lss_lon.u0) |> collect
x_labels = deleteat!(x_labels_lon, findfirst(isequal(:h), x_labels_lon))
y_labels = deleteat!(y_labels_lon, findfirst(isequal(:h), y_labels_lon))
u_labels = u_labels_lon
# x_labels = x_labels_lon
# y_labels = y_labels_lon
# u_labels = u_labels_lon

P_lss = submodel(P_lss_lon; x = x_labels, u = u_labels, y = y_labels)
P_nss = named_ss(P_lss);

In [ ]:
dampreport(P_nss_lon)
dampreport(P_nss)

We see that removing $h$ as a state eliminates an extremely slow pole, probably related to the effect of altitude on engine output. The remaining poles are virtually unaffected. Also, it significantly improves the condition of the dynamics matrix:

In [ ]:
cond(P_nss_lon.A) |> display
cond(P_nss.A) |> display

But a much more important reason for removing $h$ is that, even if the dynamics matrix is still invertible, the block $H_x F^{-1} G$ in the feedforward gain matrix required for $\theta$ and $EAS$ tracking is not. This is perfectly understandable: we cannot have an arbitrary combination of $\theta$ and $EAS$ and still maintain constant altitude. In other words, keeping $h$ in the state vector produces quasi-static rather than static equilibrium, and therefore we would need partitioning the state to handle it. Note that this issue is only related to the selection of the command variables $\theta$ and $EAS$, not to the system's dynamics. If we picked instead $h$ and $EAS$, we could have true static equilibrium. Thus, for this design we will use the MIMO system without $h$ as our plant baseline. Let's start by designing a full-state feedback regulator for the MIMO system.

## 1. LQR Design

In [ ]:
F = P_lss.A
G = P_lss.B
Hx = P_lss.C[(:θ, :EAS), :]
Hu = P_lss.D[(:θ, :EAS), :]
# display(Hx)
# display(Hu)

n_y, n_x = size(Hx) #number of command variables and states

Assemble the augmented system for LQR design by appending integrator blocks.

In [ ]:
F_aug = [F zeros(n_x, n_y); Hx zeros(n_y, n_y)]
G_aug = [G; Hu]
Hx_aug = [Hx zeros(n_y, n_y)]
Hu_aug = Hu

P_aug = ss(F_aug, G_aug, Hx_aug, Hu_aug)
controllability(F_aug, G_aug)


Design LQR and check results.

In [912]:
@unpack x0, u0 = P_lss
@unpack v_x, v_z = x0
v_norm = norm([v_x, v_z])

diagQ = ComponentVector(q = 2, θ = 40, v_x = 2/v_norm, v_z = 1/v_norm,
                        α_filt = 0, ω_eng = 0, ele_v = 0, ele_p = 0, thr_v = 0, thr_p = 0,
                        θ_int = 10, EAS_int = 0.01)

diagR = ComponentVector(ele_cmd = 0.2, thr_cmd = 0.005)

Q = ComponentMatrix(diagm(diagQ), Axis(x_labels), Axis(x_labels))
R = ComponentMatrix(diagm(diagR), Axis(x_labels), Axis(x_labels))

#compute gain matrix
C_aug = lqr(P_aug, Q, R)

#extract system state and integrator blocks
C_x = C_aug[:, 1:n_x]
C_ξ = C_aug[:, n_x+1:end]
display(C_ξ)

# quickly construct the closed-loop augmented system to check the resulting
# eigenvalues, we'll do the connections properly later
F_aug_cl = F_aug - G_aug * C_aug
P_aug_cl = ss(F_aug_cl, G_aug, Hx_aug, Hu_aug)

dampreport(P_aug)
dampreport(P_aug_cl)

Define all gain block and integrator systems:

In [ ]:
A = [F G; Hx Hu]
B = inv(A)
B_12 = B[1:n_x, n_x+1:end]
B_22 = B[n_x+1:end, n_x+1:end]

C_fbk = C_x
C_fbk_ss = named_ss(ss(C_fbk), u = x_labels, y = [:e_fbk, :t_fbk])

C_fwd = B_22 + C_x * B_12
C_fwd_ss = named_ss(ss(C_fwd), u = [:θ_fwd_in, :EAS_fwd_in], y = [:e_fwd, :t_fwd])

C_int = C_ξ
C_int_ss = named_ss(ss(C_int), u = [:θ_int_out, :EAS_int_out], y = [:e_int, :t_int])

int_ss = named_ss(ss(tf(1, [1,0])) .* I(2),
                    x = [:ξ_θ, :ξ_EAS],
                    u = [:θ_int_in, :EAS_int_in],
                    y = [:θ_int_out, :EAS_int_out]);

nothing

# display(C_fbk_ss)
# display(C_fwd_ss)
# display(C_int_ss)
# display(int_ss)

Define the properly labelled junctions and connections:

In [ ]:
θ_sum = sumblock("θ_int_in = θ_1 - θ_dmd1")
EAS_sum = sumblock("EAS_int_in = EAS_1 - EAS_dmd1")
elevator_sum = sumblock("e_cmd = e_fwd - e_fbk - e_int")
throttle_sum = sumblock("t_cmd = t_fwd - t_fbk - t_int")

θ_dmd_splitter = splitter(:θ_dmd, 2)
EAS_dmd_splitter = splitter(:EAS_dmd, 2)

test_connections = vcat(
    Pair.(C_fbk_ss.u, C_fbk_ss.u),
    :e_fwd => :e_fwd,
    :t_fwd => :t_fwd,
    :e_fbk => :e_fbk,
    :t_fbk => :t_fbk,
    :e_int => :e_int,
    :t_int => :t_int,
    :θ_int_in => :θ_int_in,
    :EAS_int_in => :EAS_int_in,
    :θ_int_out => :θ_int_out,
    :EAS_int_out => :EAS_int_out,
    :θ => :θ_1,
    :EAS => :EAS_1,
    :θ_dmd1 => :θ_dmd1,
    :EAS_dmd1 => :EAS_dmd1,
    :θ_dmd2 => :θ_fwd_in,
    :EAS_dmd2 => :EAS_fwd_in,
    :e_cmd => :elevator_cmd,
    :t_cmd => :throttle_cmd,
    )


connections = vcat(test_connections)

P_nss_cl = connect([P_nss, int_ss, C_fwd_ss, C_fbk_ss, C_int_ss,
                    θ_sum, EAS_sum, elevator_sum, throttle_sum,
                    θ_dmd_splitter, EAS_dmd_splitter], connections;
                    w1 = [:θ_dmd, :EAS_dmd], z1 = vcat(y_labels, [:θ_int_out, :EAS_int_out]))

dampreport(P_aug_cl)
dampreport(P_nss_cl)


In [ ]:
θdmd2θ = P_nss_cl[:θ, :θ_dmd]
θdmd2e = P_nss_cl[:elevator_cmd, :θ_dmd]
step(θdmd2θ, 10) |> stepinfo |> display
step(θdmd2θ, 10) |> plot
# step(θdmd2e, 10) |> plot!

In [ ]:
vdmd2v = P_nss_cl[:EAS, :EAS_dmd]
vdmd2t = P_nss_cl[:throttle_cmd, :EAS_dmd]
step(vdmd2v, 10) |> stepinfo |> display
step(vdmd2v, 10) |> plot
step(vdmd2t, 10) |> plot!

In [ ]:
marginplot(θdmd2θ)

## 3. Pitch Rate Tracker

Now, on top of the $\theta$ + $EAS$ tracker we can provide a pitch rate tracker by adding an integrator upstream of the $\theta_{dmd}$ input:

In the actual implementation, $\dot{\theta}$ should not be computed directly as $\dot{\theta} = q_{dmd}$ but through the kinematic relation:
$$ q(\dot{\theta}, r, \phi) = \dfrac{1}{\cos \phi}(\dot{\theta} + r \tan \phi) $$
$$ \dot{\theta}_{dmd} = q_{dmd} \cos \phi - r \tan \phi$$